# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f7e00261ca0>
├── 'a' --> tensor([[-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405]])
└── 'x' --> <FastTreeValue 0x7f7e00261490>
    └── 'c' --> tensor([[-0.7099,  0.1911, -2.1004,  1.2285],
                        [ 0.2755,  0.4070, -0.9535, -0.5985],
                        [-1.1073, -0.0925, -0.8500,  0.1077]])

In [4]:
t.a

tensor([[-0.3574, -0.7032,  0.6529],
        [-1.1286, -0.5080,  0.1405]])

In [5]:
%timeit t.a

75.7 ns ± 1.24 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f7e00261ca0>
├── 'a' --> tensor([[ 0.3896, -0.4968, -0.6955],
│                   [-0.4664, -0.3744, -1.9973]])
└── 'x' --> <FastTreeValue 0x7f7e00261490>
    └── 'c' --> tensor([[-0.7099,  0.1911, -2.1004,  1.2285],
                        [ 0.2755,  0.4070, -0.9535, -0.5985],
                        [-1.1073, -0.0925, -0.8500,  0.1077]])

In [7]:
%timeit t.a = new_value

84.4 ns ± 1.03 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405]]),
    x: Batch(
           c: tensor([[-0.7099,  0.1911, -2.1004,  1.2285],
                      [ 0.2755,  0.4070, -0.9535, -0.5985],
                      [-1.1073, -0.0925, -0.8500,  0.1077]]),
       ),
)

In [10]:
b.a

tensor([[-0.3574, -0.7032,  0.6529],
        [-1.1286, -0.5080,  0.1405]])

In [11]:
%timeit b.a

70.8 ns ± 1.42 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.7762, -1.7316,  0.0786],
               [-2.2128,  0.1379,  0.6481]]),
    x: Batch(
           c: tensor([[-0.7099,  0.1911, -2.1004,  1.2285],
                      [ 0.2755,  0.4070, -0.9535, -0.5985],
                      [-1.1073, -0.0925, -0.8500,  0.1077]]),
       ),
)

In [13]:
%timeit b.a = new_value

628 ns ± 8.67 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.03 µs ± 16.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14 µs ± 292 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

294 µs ± 9.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

274 µs ± 7.75 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f7d501646a0>
├── 'a' --> tensor([[[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]],
│           
│                   [[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]],
│           
│                   [[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]],
│           
│                   [[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]],
│           
│                   [[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]],
│           
│                   [[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]],
│           
│                   [[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]],
│           
│                   [[-0.3574, -0.7032,  0.6529],
│                    [-1.1286, -0.5080,  0.1405]]])
└── 'x' --> <FastTreeValue 0x7f7d50164400>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

48.7 µs ± 800 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f7d501645b0>
├── 'a' --> tensor([[-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405],
│                   [-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405],
│                   [-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405],
│                   [-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405],
│                   [-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405],
│                   [-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405],
│                   [-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405],
│                   [-0.3574, -0.7032,  0.6529],
│                   [-1.1286, -0.5080,  0.1405]])
└── 'x' --> <FastTreeValue 0x7f7dd82c3b80>
    └── 'c' --> tensor([[-0.7099,  0.1911, -2.1004,  1.2285],
                        [ 0.2755,  0.4070, -0.9535, -0.5985],
                 

In [23]:
%timeit t_cat(trees)

42.7 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

87.5 µs ± 936 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]],
       
               [[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]],
       
               [[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]],
       
               [[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]],
       
               [[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]],
       
               [[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]],
       
               [[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]],
       
               [[-0.3574, -0.7032,  0.6529],
                [-1.1286, -0.5080,  0.1405]]]),
    x: Batch(
           c: tensor([[[-0.7099,  0.1911, -2.1004,  1.2285],
                       [ 0.2755,  0.4070, -0.9535, -0.5985],
                       [-1.1073, -0.0925, -0.8500,  0.1077]],
         

In [26]:
%timeit Batch.stack(batches)

108 µs ± 1.2 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405],
               [-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405],
               [-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405],
               [-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405],
               [-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405],
               [-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405],
               [-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405],
               [-0.3574, -0.7032,  0.6529],
               [-1.1286, -0.5080,  0.1405]]),
    x: Batch(
           c: tensor([[-0.7099,  0.1911, -2.1004,  1.2285],
                      [ 0.2755,  0.4070, -0.9535, -0.5985],
                      [-1.1073, -0.0925, -0.8500,  0.1077],
                      [-0.7099,  0.1911, -2.1004,  1.2285],
                      [ 0.2755,  

In [28]:
%timeit Batch.cat(batches)

189 µs ± 1.5 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

587 µs ± 6.33 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
